In [1]:
import mysql.connector
import pandas as pd
from decimal import Decimal
from datetime import date, datetime
import pprint
import math
import boto3
from boto3.dynamodb.conditions import Key, Attr
import csv

In [2]:
#colocar aqui os dados necessarios para o filtro
ano = "(2022)"
mes = "(2)"
cliente = "(362, 1116, 1601, 6293, 8267, 19210, 36148, 36157)"
#VERIFICAR OS FERIADOS E OS DIAS QUE DEVEM SER ADICIONADOS LA EMBAIXO NO CÓDIGO. 
#PARA MAIS DE UM FERIADO, ADD IFS UM ABAIXO DO OUTRO
#(o comentario na parte do codigo esta igual ao acima)

In [3]:
#conexão com o banco de dados para capturar as solicitações desejadas
con = mysql.connector.connect(host="#######", 
                              user="#######", passwd="#######", db="#######")
cursor = con.cursor(buffered=True)
query = "select distinct(s.id) from solicitacao s inner join pretendente p on p.solicitacao_id = s.id inner join pretendente_produto pp on pp.pretendente_id = p.id where month(pp.payment_at) in "+mes+" and year(pp.payment_at) in "+ano+" and s.cliente_id in "+cliente+""
cursor.execute(query)
solicitacoes = []
for resultado in cursor:
    solicitacoes.append(resultado[0])

In [4]:
#Fazendo conexao com o banco aws para pegar somente as solicitacoes retornadas no banco mysql
dynamodb = boto3.resource('dynamodb',
                          region_name='us-east-1',
                          aws_access_key_id='#######',
                          aws_secret_access_key='######')
table = dynamodb.Table('evaluation-report-data-prod')
responses = []
f = Attr('data.pretendente.id').ne(0)
for i in range(len(solicitacoes)):
    responses.append(table.query(FilterExpression=f,
                                 KeyConditionExpression=Key('solicitation_id').eq(solicitacoes[i])))

In [5]:
tipos_renda = {1:"Renda Não Informada", 2:"Funcionário Público (Estatutário)", 3:"Funcionário Público (CLT)",
              4:"Empresário", 5:"Profissional Liberal ou Autônomo", 6:"Aposentado ou Pensionista", 7:"Renda de Aluguel",
              8:"Pensão Alimentícia ou Judicial", 9:"Estagiário ou Bolsista", 11:"Funcionário Privado Registrado (CLT)",
              12:"Militar", 13:"Limite do Cartão de Crédito", 14:"Outro", 15:"Extratos Bancários"}
relatorio = {}
i = 1
for t in tipos_renda:
    if(i==10):
        i = i + 1
    relatorio[tipos_renda[i]] = {}
    relatorio[tipos_renda[i]]['TOTAL'] = 0 
    relatorio[tipos_renda[i]]['TEMPO ATENDIMENTO DIAS'] = {}
    relatorio[tipos_renda[i]]['TEMPO ATENDIMENTO DIAS']['0'] = 0
    relatorio[tipos_renda[i]]['TEMPO ATENDIMENTO MINUTOS'] = {}
    relatorio[tipos_renda[i]]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 15'] = 0
    relatorio[tipos_renda[i]]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 30'] = 0
    relatorio[tipos_renda[i]]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 40'] = 0
    relatorio[tipos_renda[i]]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 50'] = 0
    relatorio[tipos_renda[i]]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 60'] = 0
    relatorio[tipos_renda[i]]['TEMPO ATENDIMENTO MINUTOS']['MAIS DE 60'] = 0
    i = i + 1
    
#criando um dicionário para pegar os tempos de resolução da análise
duracoes = {}
i = 1
for t in tipos_renda:
    if(i==10):
        i = i + 1
    duracoes[tipos_renda[i]] = []
    i = i + 1
duracoes['TOTAL'] = []

In [6]:
for i in range(len(responses)):
    
    for j in range(len(responses[i]['Items'])):
        #nem todas as solicitações tem fc renda. então só pega aquelas que tem análise de fc renda
        if 'renda' in responses[i]['Items'][j]['data']['pretendente']:
            if 'analises' in responses[i]['Items'][j]['data']['pretendente']['renda'] and len(responses[i]['Items'][j]['data']['pretendente']['renda']['analises']) > 0 and isinstance(responses[i]['Items'][j]['data']['pretendente']['renda']['analises'][0]['data_fim'], str):

                #printando o cliente id para verificacao
                print('Solicitação: ', solicitacoes[i])
                print("Cliente: ", responses[i]['Items'][j]['data']['extra']['client_id'])

                #transformando a data que vem do json para datetime do pandas
                inicio_analise  = pd.to_datetime(responses[i]['Items'][j]['data']['pretendente']['renda']['analises'][0]['data_inicio'])    
                fim_analise = pd.to_datetime(responses[i]['Items'][j]['data']['pretendente']['renda']['analises'][0]['data_fim'])

                #pegando a data de inicio da analise e estabelecendo as horas de expediente
                inicio_expediente = inicio_analise
                inicio_expediente = inicio_expediente.replace(hour = 9)
                inicio_expediente = inicio_expediente.replace(minute = 0)
                inicio_expediente = inicio_expediente.replace(second = 0)

                fim_expediente = inicio_analise
                fim_expediente = fim_expediente.replace(hour = 17)
                fim_expediente = fim_expediente.replace(minute = 30)
                fim_expediente = fim_expediente.replace(second = 0)

                #pegando o nome do tipo de renda
                tipo_renda = responses[i]['Items'][j]['data']['pretendente']['renda']['analises'][0]['origem_renda_principal_titulo']

                #pegando a duração de dias e de horas das datas de fim e início da análise
                duracao = fim_analise - inicio_analise

                #imprimindo nome e datas para verificação e testes
                print(tipo_renda)
                print("Data e hora de início: ", inicio_analise)
                print("Data e hora de fim: ", fim_analise)
                print("Duração: " + str(duracao.days) + " dias / " + str(round(duracao.seconds/60)) + " minutos")

                # se a data de fim for maior que a data de início,
                # é provável que o início ou fim da análise tenha sido após ou antes do expediente (9 as 17:30)
                # ou se os dois foram dentro do horário, só conta a diferença de tempo dentro do horário 
                # de expediente e não da diferença inteira de horas.
                if fim_analise.date() > inicio_analise.date():

                    print('** data de fim da análise maior que de início **')

                    # zerando a duracao no tipo timedelta    
                    duracao = inicio_expediente - inicio_expediente    

                    #VERIFICAR OS FERIADOS E OS DIAS QUE DEVEM SER ADICIONADOS:
                    #PARA MAIS DE UM FERIADO, ADD IFS UM ABAIXO DO OUTRO
                    if inicio_analise.day == 24 or inicio_analise.day == 31 :
                        print('iniciou num feriado!')
                        add_days = '3 days' 

                    # se a análise foi iniciada segunda,terça,quarta,quinta ou domingo 
                    # só começa a contar no dia seguinte
                    elif inicio_analise.weekday() in [0,1,2,3] or inicio_analise.weekday() == 6:
                        add_days = '1 days'

                    # se a análise foi iniciada na sexta
                    # só começa a contar a partir de segunda
                    elif inicio_analise.weekday() == 4:
                        add_days = '3 days'

                    # se a análise foi iniciada no sábado
                    elif inicio_analise.weekday() == 5:
                        add_days = '2 days'

                    # se a análise iniciou antes do fim do expediente
                    # se não, precisamos verificar primeiro se iniciou antes do horário de expediente
                    # para transformar no inicio do expediente, fazer a diferença entre o fim e o início
                    # do expediente, adicionar a quantidade de dias e fazer a comparação.
                    # se não iniciou, faz apenas a diferença
                    # if inicio_analise < fim_expediente:
                    # if inicio_analise <= inicio_expediente:
                    # duracao = fim_expediente - inicio_expediente
                    # else:
                    # duracao = fim_expediente - inicio_analise

                    # agora, adicionamos os dias correspondentes ao inicio do expediente
                    inicio_expediente = inicio_expediente + pd.Timedelta(add_days)

                    # agora, calculamos a duração com a data de conclusão 
                    # se a data de inicio for maior que a de fim, a duração é zero 
                    # exemplo: iniciou na sexta e terminou no sábado, porém adicionamos três dias na sexta e 
                    # ficou maior. se não, faz a diferença
                    if inicio_expediente > fim_analise:
                        duracao = pd.Timedelta('0 days')
                    else:
                        duracao = duracao + (fim_analise - inicio_expediente)

                    print("Duração APÓS AJUSTE: " + str(duracao.days) + " dias / " + str(round(duracao.seconds/60)) + " minutos")

                # se a data de início for igual a data de fim
                else:

                    print('** data de fim da análise igual de início **')

                    # para fim de análise, consideramos fim de expediente 18hrs
                    fim_expediente_conclusao = fim_expediente.replace(hour = 18)
                    fim_expediente_conclusao = fim_expediente.replace(minute = 30)

                    # se a analise foi iniciada e finalizada fora do horário de expediente, 
                    # entao a duração é zero
                    if (inicio_analise < inicio_expediente and fim_analise <= inicio_expediente) or (inicio_analise > fim_expediente and fim_analise > fim_expediente):
                        duracao = inicio_expediente - inicio_expediente 

                    # se a analise foi iniciada antes do expediente, a duração é calculada 
                    # começando de 9h (inicio do expediente)
                    elif inicio_analise < inicio_expediente and fim_analise < fim_expediente_conclusao:
                        duracao = fim_analise - inicio_expediente

                    # se a analise foi finalizada depois do expediente, a duração é calculada 
                    # com o horário do fim do expediente 
                    elif (inicio_analise >= inicio_expediente and inicio_analise < fim_expediente) and fim_analise > fim_expediente_conclusao:
                        duracao = fim_expediente_conclusao - inicio_analise

                print('---------')
                print('')

                if(pd.notnull(duracao.days)):   

                    #somando +1 ao total
                    relatorio[tipo_renda]['TOTAL'] += 1

                    #verificando intervalo de minutos da solicitacao
                    minutes = duracao.seconds/60

                    #adicionando +1 ao intervalo calculado
                    if minutes <= 15:
                        relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 15'] += 1

                    elif minutes > 15 and minutes <= 30:
                        relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 30'] += 1

                    elif minutes > 30 and minutes <= 40:
                        relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 40'] += 1

                    elif minutes > 40 and minutes <= 50:
                        relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 50'] += 1

                    elif minutes > 50 and minutes <= 60:
                        relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 60'] += 1

                    elif minutes > 60:
                        relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['MAIS DE 60'] += 1

                    #se existe então soma +1 para contar quantas solicitações tem com aquela 
                    #diferença de dias
                    if str(math.floor(minutes/480)) not in relatorio[tipo_renda]['TEMPO ATENDIMENTO DIAS']:
                        relatorio[tipo_renda]['TEMPO ATENDIMENTO DIAS'][str(math.floor(minutes/480))] = 0
                    relatorio[tipo_renda]['TEMPO ATENDIMENTO DIAS'][str(math.floor(minutes/480))] += 1 

                    duracoes[tipo_renda].append(duracao.seconds)
                    duracoes['TOTAL'].append(duracao.seconds)

Solicitação:  64029
Cliente:  1116
Limite do Cartão de Crédito
Data e hora de início:  2022-01-27 15:19:05
Data e hora de fim:  2022-01-27 15:27:17
Duração: 0 dias / 8 minutos
** data de fim da análise igual de início **
---------

Solicitação:  64736
Cliente:  362
Renda Não Informada
Data e hora de início:  2022-02-02 10:32:09
Data e hora de fim:  2022-02-02 10:50:33
Duração: 0 dias / 18 minutos
** data de fim da análise igual de início **
---------

Solicitação:  64736
Cliente:  362
Renda Não Informada
Data e hora de início:  2022-02-02 10:32:09
Data e hora de fim:  2022-02-02 10:50:54
Duração: 0 dias / 19 minutos
** data de fim da análise igual de início **
---------

Solicitação:  64773
Cliente:  362
Extratos Bancários
Data e hora de início:  2022-02-02 12:03:34
Data e hora de fim:  2022-02-02 12:22:37
Duração: 0 dias / 19 minutos
** data de fim da análise igual de início **
---------

Solicitação:  64773
Cliente:  362
Extratos Bancários
Data e hora de início:  2022-02-02 12:03:34


Data e hora de fim:  2022-02-28 09:52:30
Duração: 1 dias / 977 minutos
** data de fim da análise maior que de início **
Duração APÓS AJUSTE: 0 dias / 52 minutos
---------

Solicitação:  68305
Cliente:  1116
Funcionário Privado Registrado (CLT)
Data e hora de início:  2022-02-28 11:45:55
Data e hora de fim:  2022-02-28 12:03:24
Duração: 0 dias / 17 minutos
** data de fim da análise igual de início **
---------

Solicitação:  68305
Cliente:  1116
Funcionário Público (Estatutário)
Data e hora de início:  2022-02-28 11:45:55
Data e hora de fim:  2022-02-28 12:05:00
Duração: 0 dias / 19 minutos
** data de fim da análise igual de início **
---------

Solicitação:  68311
Cliente:  1116
Extratos Bancários
Data e hora de início:  2022-02-28 12:45:41
Data e hora de fim:  2022-02-28 12:53:13
Duração: 0 dias / 8 minutos
** data de fim da análise igual de início **
---------

Solicitação:  68311
Cliente:  1116
Extratos Bancários
Data e hora de início:  2022-02-28 12:45:41
Data e hora de fim:  2022-

In [7]:
csv_columns = ['TIPO RENDA', 'TOTAL', 'DIAS (0)', 'MIN. (<15)', 'MIN. (<30)', 'MIN. (<40)', 'MIN. (<50)', 'MIN. (<60)', 'MIN. (>60)', 'MÉDIA MIN.']
with open('relatorio-tempo-medio-fcrenda.csv', 'w', newline='', encoding='utf-8') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter.writerow(csv_columns)
    for key in relatorio:
        listaRow = list()
        listaRow.append(key)
      
        listaRow.append(relatorio[key]['TOTAL'])
        listaRow.append(relatorio[key]['TEMPO ATENDIMENTO DIAS']['0'])
        listaRow.append(relatorio[key]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 15'])
        listaRow.append(relatorio[key]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 30'])
        listaRow.append(relatorio[key]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 40'])
        listaRow.append(relatorio[key]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 50'])
        listaRow.append(relatorio[key]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 60'])
        listaRow.append(relatorio[key]['TEMPO ATENDIMENTO MINUTOS']['MAIS DE 60'])
        if(len(duracoes[key]) > 0):
            listaRow.append(round(sum(duracoes[key])/len(duracoes[key])/60))
        else:
            listaRow.append('-')
        
        
        spamwriter.writerow(listaRow)
        
print('Script executado com sucesso!')            

Script executado com sucesso!
